In [ ]:
%load_ext autoreload
%autoreload 2
from config import username, password, ellevation_host, ellevation_username, ellevation_password
from modules.login_module import *
from modules.download_files_module import *
from modules.unit_testing import TestFileProcessing
from modules.data_transformation import *
from modules.post_download_change import *
from modules.sql_query_module import *
from modules.sftp_ops import *
from modules.logging_metadata import *

import os
import pandas as pd
import logging
import time
from datetime import datetime
from modules.sql_query_module import SQL_query
import urllib
import sqlalchemy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
today_date = datetime.now()
formatted_month_day_year = today_date.strftime("%m_%d_%y")
# formatted_month_day_year = '06_17_24' #temporarily in here for up to date send
download_directory = os.getcwd() + f'\\file_downloads\\elpac\\{formatted_month_day_year}'
pd.set_option('display.max_columns', None)

logger = JobLogger(process_name='ELPAC_SBAC_Results_Selenium', 
                   job_name='ELPAC_SBAC_Results_Selenium', 
                   job_type='python')

logging.basicConfig(filename='ELPAC_SBAC_results.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S',force=True)
logging.info('\n\n-------------ELPAC_SBAC_results new instance log')


# Set up Chrome options
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : download_directory,
         'profile.default_content_setting_values.automatic_downloads': 1,
         'profile.content_settings.exceptions.automatic_downloads.*.setting': 1}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)

#instantiate SFTP conn with Ellevation
sftp_conn_ellevation = SFTPConnection(
    host=ellevation_host,
    username=ellevation_username,
    password=ellevation_password,
    use_pool=False
)


def selenium_process(SY):

    logIn(username, password, driver)
    launch_to_homescreen(driver)

    # ---------------------------------------SBAC & ELPAC Files Request and Download-------
    # Call the function, school report names variable is called for just school name. MUst occur in this order for Selenium
    #Equivalent of Student Score Data File
    SBAC_output = SBAC_package_func(driver, SY, 'Tested', formatted_month_day_year)
    ELPAC_output = ELPAC_package_func(driver, SY, 'Tested', formatted_month_day_year)

    return(SBAC_output, ELPAC_output)


# # -----------------------------------------------Send over new records------------------------

def send_to_sql(frame, file_name):

    naming_convention = f'{file_name}_Scores'

    dtypes, table_cols = SQL_query.get_dtypes(frame, 'DataTeamSandbox', naming_convention)

    #Update the master table with a full replace, after assessing todays incoming records by each table
    try:
        frame.to_sql(naming_convention, schema='dbo', con = SQL_query.engine, if_exists = 'replace', index = False, dtype=dtypes)
        logging.info(f"Sent data - {len(frame)} records to master table {naming_convention}")
    except Exception as e:
        logging.info(f'Unable to send data to {naming_convention} due to \n {e}')
    
    #Duplicates need to be dropped within stored proc for next year

#Notes
#If files are not downloaded all the way through do not let the send occur on the append. 
#Set up stored procedure to drop duplicates when sending to SQL

def main():

    sbac_local_dir = fr'C:\Users\samuel.taylor\Desktop\Python_Scripts\Manual_Triggers\ELPAC_SBAC_Results_Selenium\file_downloads\sbac\{formatted_month_day_year}'
    elpac_local_dir = fr'C:\Users\samuel.taylor\Desktop\Python_Scripts\Manual_Triggers\ELPAC_SBAC_Results_Selenium\file_downloads\elpac\{formatted_month_day_year}'
    sbac_pdrive_dir = fr'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing\sbac_{formatted_month_day_year}'
    elpac_pdrive_dir = fr'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing\elpac_{formatted_month_day_year}'

    #clear out download directories prior in the case or re-runs
    # Example usage
    for i in [sbac_local_dir, elpac_local_dir, sbac_pdrive_dir, elpac_pdrive_dir]:
        empty_directory(i)
        create_directory(i)

    SBAC_output, ELPAC_output = selenium_process('2024')

    # --------------------------------Unzip the XLSX Files and Move them to the P-Drive, Additional Unit Test---------------Path - 'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing'

    SBAC_output = unzip_move_and_unit(SBAC_output, 'sbac', formatted_month_day_year)
    ELPAC_output = unzip_move_and_unit(ELPAC_output, 'elpac', formatted_month_day_year)

    # # ---------------------------------POST SELENIUM PROCESS, STACKING & SENDING FILES----------------------------------
    sbac_stack = stack_files(sbac_pdrive_dir, 'CAASPP') #This is where files are raw and stacked before transformation
    elpac_stack = stack_files(elpac_pdrive_dir, 'ELPAC') #Green Dot Schools are pulled out of master file

    # # -----------------------------Where the normalization of the dataframes occur, column changing & mapping------------------
    elpac = get_elpac_import(elpac_stack, 'ELPAC')
    sbac = get_sbac_import(sbac_stack, 'SBAC')  #For some reason, raw ELPAC file does not have LocalStudentID or studentnumber present for SBAC ELA & Math overall 
    cast = get_cast_import(sbac_stack, 'CAST')

    #For Helens Ellevation Pickup, send to same dir as individual files the stack in stacked_files dir
    #Use this as route to drop raw files
    send_stacked_csv(elpac, 'ELPAC', formatted_month_day_year) 
    send_stacked_csv(sbac, 'SBAC', formatted_month_day_year)
    send_stacked_csv(cast, 'CAST', formatted_month_day_year)

    # Export the local replicated files to Clevers SFTP
    SFTP_export_dir_to_SFTP(local_dir=os.getcwd() + '\\clever_iota_file_transfer',
                remote_dir='/data',  #root dir on clevers sftp
                sftp = sftp_conn_ellevation)
    
    send_to_sql(elpac, 'ELPAC')
    send_to_sql(sbac, 'SBAC')
    send_to_sql(cast, 'CAST')


try:
    main()
    logging.info('Process was a success')
    logger.log_job('Success')
    logger.send_frame_to_SQL()
except Exception as e:     
    logging.info(f'Process failed due to the following: {e}')
    logger.log_job('Failure')
    logger.send_frame_to_SQL()

# Sam By Enrolled Request

In [ ]:
%load_ext autoreload
%autoreload 2
from config import username, password
from modules.login_module import *
from modules.download_files_module import *
from modules.unit_testing import TestFileProcessing
from modules.data_transformation import *
from modules.post_download_change import *
from modules.sql_query_module import *
from modules.creating_subscores_cast import *
from modules.creating_subscores_cast import *
from modules.creating_subscores_math import *
from modules.creating_subscores_ela import *


from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException, NoSuchWindowException
from selenium.webdriver.common.action_chains import ActionChains
import os
import pandas as pd
import logging
from datetime import datetime
from modules.sql_query_module import SQL_query
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
today_date = datetime.now()
formatted_month_day_year = today_date.strftime("%m_%d_%y")
download_directory = os.getcwd() + f'\\file_downloads\\elpac\\{formatted_month_day_year}'
pd.set_option('display.max_columns', None)

logging.basicConfig(filename='ELPAC_SBAC_results.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S',force=True)
logging.info('\n\n-------------ELPAC_SBAC_results new instance log')


#clear out download directories prior in the case or re-runs
# Example usage

sbac_local_dir = fr'C:\Users\samuel.taylor\Desktop\Python_Scripts\Manual_Triggers\ELPAC_SBAC_Results_Selenium\file_downloads\sbac\{formatted_month_day_year}'
elpac_local_dir = fr'C:\Users\samuel.taylor\Desktop\Python_Scripts\Manual_Triggers\ELPAC_SBAC_Results_Selenium\file_downloads\elpac\{formatted_month_day_year}'
sbac_pdrive_dir = fr'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing\sbac_{formatted_month_day_year}'
elpac_pdrive_dir = fr'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing\elpac_{formatted_month_day_year}'


for i in [sbac_local_dir, elpac_local_dir, sbac_pdrive_dir, elpac_pdrive_dir]:
    empty_directory(i)
    create_directory(i)

# Set up Chrome options
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : download_directory,
         'profile.default_content_setting_values.automatic_downloads': 1,
         'profile.content_settings.exceptions.automatic_downloads.*.setting': 1}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)


def selenium_process(SY):

    logIn(username, password, driver)
    launch_to_homescreen(driver)

    # ---------------------------------------SBAC & ELPAC Files Request and Download-------
    # Call the function, school report names variable is called for just school name. MUst occur in this order for Selenium
    #Equivalent of Student Score Data File
    SBAC_output = SBAC_package_func(driver, SY, 'Enrolled', formatted_month_day_year)
    # ELPAC_output = ELPAC_package_func(driver, SY, 'Enrolled', formatted_month_day_year)

    return(SBAC_output)

SBAC_output = selenium_process('2023')

# --------------------------------Unzip the XLSX Files and Move them to the P-Drive, Additional Unit Test---------------Path - 'P:\Knowledge Management\Ellevation\Data Sent 2023-24\State Testing'

# SBAC_output = unzip_move_and_unit(SBAC_output, 'sbac', formatted_month_day_year)
# ELPAC_output = unzip_move_and_unit(ELPAC_output, 'elpac', formatted_month_day_year)

# # # # ---------------------------------POST SELENIUM PROCESS, STACKING & SENDING FILES----------------------------------

# sbac_stack = stack_files(sbac_pdrive_dir, 'CAASPP') #This is where files are raw and stacked before transformation
# elpac_stack = stack_files(elpac_pdrive_dir, 'ELPAC') #Green Dot Schools are pulled out of master file

# # # -----------------------------Where the normalization of the dataframes occur, column changing & mapping------------------
# # elpac = get_elpac_import(elpac_stack, 'ELPAC')
# sbac = get_sbac_import(sbac_stack, 'SBAC')  #For some reason, raw ELPAC file does not have LocalStudentID or studentnumber present for SBAC ELA & Math overall 
# cast = get_cast_import(sbac_stack, 'CAST')

# sbac.to_csv('P:\Knowledge Management\Temp\SBAC_Enrolled_updated_2023.csv', index=False)
# cast.to_csv('P:\Knowledge Management\Temp\CAST_Enrolled_updated_2023.csv', index=False)

# # # -----------------------------Where the normalization of the dataframes occur, column changing & mapping------------------
# elpac = get_elpac_import(elpac_stack, 'ELPAC')
# sbac = get_sbac_import(sbac_stack, 'SBAC')

#Ensure by enrolled 2022 (these download straight away)
#Enrolled 2023 (Has to be loaded first, and downloaded on seperate screen)

# --------------------For the 2022 Enrolled Request----------------------------
#MOve files form ELPAC to SBAC manually first
# unzip_files_in_same_dir('sbac', formatted_month_day_year)

# sbac_stack = stack_files(sbac_local_dir, 'CAASPP') #This is where files are raw and stacked before transformation

# sbac = get_sbac_import(sbac_stack, 'SBAC')  #For some reason, raw ELPAC file does not have LocalStudentID or studentnumber present for SBAC ELA & Math overall

# sbac.to_csv('P:\Knowledge Management\Temp\SBAC_Enrolled_2022_updated.csv', index=False)
